In [ ]:
# 下記セルを実行すると、Googleドライブへの接続を求められます。
# Googleアカウントにログインして進めてください。
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
path = f'/content/drive/MyDrive/{working_dir}/本章/1章'
os.chdir(path)

# １章 分析に向けた準備を行う１０本ノック

ここでは、あるピザチェーンの売上の推移を分析していきます。  
データの中身を理解し、分析しやすい形に加工していくプロセスを学びます。

### ノック１：データを全て読み込んでみよう

In [ ]:
import pandas as pd
m_store = pd.read_csv('m_store.csv')
m_store

In [ ]:
len(m_store)

In [ ]:
m_store.head()

In [ ]:
m_area = pd.read_csv('m_area.csv')
m_area

In [ ]:
tbl_order_4 = pd.read_csv('tbl_order_202004.csv')
tbl_order_4

### ノック２：データを結合（ユニオン）してみよう

In [ ]:
tbl_order_5 = pd.read_csv('tbl_order_202005.csv')
tbl_order_5

In [ ]:
order_all = pd.concat([tbl_order_4, tbl_order_5], ignore_index=True)
order_all

In [ ]:
len(order_all) == len(tbl_order_4) + len(tbl_order_5)

### ノック３：フォルダ内のファイル名を一覧化してみよう

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
# 全てのファイルを一覧表示する
os.listdir(current_dir)

In [ ]:
tbl_order_file = os.path.join(current_dir, 'tbl_order_*.csv')
tbl_order_file

In [ ]:
# 特定のファイルを一覧表示する
import glob
tbl_order_files = glob.glob(tbl_order_file)
tbl_order_files

### ノック４：複数データを結合（ユニオン）してみよう

In [ ]:
# 繰り返しの前に1個だけ処理して確認したい場合
order_all = pd.DataFrame()
file = tbl_order_files[0]
order_data = pd.read_csv(file)
print(f'{file}:{len(order_data)}')
order_all = pd.concat([order_all, order_data], ignore_index=True)
order_all

In [ ]:
# 処理を繰り返す場合
order_all = pd.DataFrame()
for file in tbl_order_files:
    order_data = pd.read_csv(file)
    print(f'{file}:{len(order_data)}')
    order_all = pd.concat([order_all, order_data], ignore_index=True)

In [ ]:
order_all

### ノック５：データの統計量を確認しよう

In [ ]:
order_all.isnull().sum()

In [ ]:
order_all.describe()

In [ ]:
order_all['total_amount'].describe()

In [ ]:
print(order_all["order_accept_date"].min())
print(order_all["order_accept_date"].max())
print(order_all["delivered_date"].min())
print(order_all["delivered_date"].max())

### ノック６：不要なデータを除外しよう

In [ ]:
order_data = order_all.loc[order_all['store_id'] != 999]
order_data

### ノック７：マスタデータを結合（ジョイン）してみよう

In [ ]:
order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data

In [ ]:
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')
order_data

### ノック８：マスタが存在しないコードに名称を設定しよう

In [ ]:
order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'
order_data

In [ ]:
order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'
order_data

### ノック９：分析基礎テーブルを出力してみよう

In [ ]:
output_dir = os.path.join(current_dir, 'output_data')
os.makedirs(output_dir, exist_ok=True)

In [ ]:
output_file = os.path.join(output_dir, 'order_data.csv')
order_data.to_csv(output_file, index=False)

### ノック１０：セルを整理して使いやすくしよう

In [ ]:
# ライブラリのインポート
import pandas as pd
import os
import glob

# ファイルの読込
m_store = pd.read_csv('m_store.csv')
m_area = pd.read_csv('m_area.csv')

# オーダーデータの読込
current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir, 'tbl_order_*.csv')
tbl_order_files = glob.glob(tbl_order_file)
order_all = pd.DataFrame()

for file in tbl_order_files:
    order_data = pd.read_csv(file)
    print(f'{file}:{len(order_data)}')
    order_all = pd.concat([order_all, order_data], ignore_index=True)

# 不要なデータを除外
order_data = order_all.loc[order_all['store_id'] != 999]

# マスタデータの結合
order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

# 名称を設定（お渡し方法）
order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'

# 名称を設定（注文状態）
order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'

# ファイルの出力
output_dir = os.path.join(current_dir, 'output_data')
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'order_data.csv')
order_data.to_csv(output_file, index=False)